
# DISTANCE MATRIX 1xN
### Developed by Bernardo Puente 

Last update: 10-Dec-2020

## Context

The Distance Matrix API is a service that provides travel distance and time for a matrix of origins and destinations. The API returns information based on the recommended route between start and end points, as calculated by the Google Maps API, and consists of rows containing duration and distance values for each pair.

The reason for this code being 1xN is that it will compute distances between a singe 1xN origin destination table. This can be usefull to compute distances from a single origin to multiple destinations, or to obtain individual distances between desired origin destination coordinates.

<img src=" https://i.stack.imgur.com/UvBdE.jpg" alt="drawing" width="400">


For more information on GOOGLE MAPS DISTANCE MATRIX API visit: https://developers.google.com/maps/documentation/distance-matrix/intro?hl=es and https://developers.google.com/maps/documentation/distance-matrix/usage-and-billing#distance-matrix-advanced


## Input
This code takes as input an excel file containing geographic coordinates (lat,long) of the origins and destinations wich we want to obtain the corresponding distances.

**Important:** There are two types of distance matrix queries *"Distance Matrx Basic"* and *"Distance Matrix Advance"*. A *Distance Matrix Basic* query will output only information regarding distances, in contrast a *Distance Matrix Advance* query will consider traffic information.

**Important:** Each query sent to the *Distance Matrix Basic API* generates elements, where the number of origins times the number of destinations equals the number of elements. The cost for this type of query is 5.00 USD per 1000, so with the given monthly 200 USD credit you have access to 40 000 *Distance Matrix Basic* queries.

**Important:** A Distance Matrix Advanced SKU is charged for a Distance Matrix API or a Maps JavaScript API’s Distance Matrix Service **that uses traffic information**. Traffic information is used when all the following apply (these are the conditions required to receive the ***duration_in_traffic*** field in the Distance Matrix response):

* The ***travel mode*** parameter is ***driving***, or is not specified (***driving*** is the default travel mode).
* The request includes a valid ***departure_time parameter***. The ***departure_time*** can be set to the current time or some time in the future. It cannot be in the past.

The cost for the *Distance Matrix Advance API* is 10.00 USD per 1000 **(twice the price for the basic API!)**, so with the given monthly 200 USD credit you have access to 20 000 *Distance Matrix Advance API* queries.

## Output
This code will output a similar file as the input file, with the original origin and destination coordinates, plus additional columns containing the resulting distances when using *Distance Matrix Basic API*, and aditional duration in traffic times when using *Distance Matrix Advance API*.

# Code Start

Import all libraries needed:

* pandas for input and output dataframe handling
* numpy to handle google maps API output in list and dict forms
* googlemaps to activate Geocoding API requests
* logging to access logger messages
* time to measure code execution times

In [1]:
import pandas as pd
import numpy as np
import googlemaps
import logging
import time

Set up logger to show desired messages in console as warinings.

In [2]:
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

Specify api key name, input/output files, and haul query hour according to Unix time.

In [3]:
api_key='YourAPI_Key_here'
file_input='INPUT DISTANCE MATRIX 1XN.xlsx'
file_output='OUTPUT DISTANCE MATRIX 1XN.xlsx'
now=int(time.time()) #Uses Unix time to current hour.
query_time=1604379600 #Use Unix time. Time set to Tue. 03 Nov 2020 5:00 am local time (Quito). Update this and use with DM Advance.
gmaps = googlemaps.Client(client_id='DistanceMatrixExercise', key=api_key)

Read input file and identify data structure.

In [4]:
Datos = pd.read_excel(file_input)
Datos.head()

,ORIGEN_ID,ORIG_LAT,ORIG_LONG,DEST_ID,DEST_LAT,DEST_LONG
0,450A,-3.445084,-79.962064,452A,-3.478560,-80.236050
1,804A,-1.050004,-80.464027,805A,-1.054404,-80.460423
2,807A,-0.949732,-80.728829,841A,-0.966395,-80.703228
3,543A,0.968458,-79.651099,111A,-0.252573,-79.165470
4,165A,-0.241837,-79.171530,111A,-0.252573,-79.165470


Test the API functionality. Check test distance result and errors.

In [5]:
start_test = time.time()
gmaps = googlemaps.Client(key=api_key)

actual_register=Datos.loc[0].tolist() #Obtain the first register of the dataframe
test_orig=(actual_register[1],actual_register[2]) #Within the actual register obtain the lat,long colums which correspond to columns [1]ORIG_LAT and [2]ORIG_LONG
test_dest=(actual_register[4],actual_register[5]) #Similar to the previous, select lat,long of the destination with correspond to columns [4]DEST_LAT [5]DEST_LONG

try:
    test_result_basic = gmaps.distance_matrix(test_orig, test_dest) #Set Distance Matrix Basic parameters
    test_result_advance = gmaps.distance_matrix(test_orig, test_dest, mode='driving', departure_time=now, traffic_model='best_guess') #Set Distance Matrix Advance parameters
    end_test=(time.time() - start_test)
    logger.warning('Distance Matrix Basic results for: \n'+str(Datos.loc[0])+'\n are: \n'+str(test_result_basic))
    logger.warning('---')
    logger.warning('Distance Matrix Advance results for: \n'+str(Datos.loc[0])+'\n are: \n'+str(test_result_advance))
    logger.warning('Verify the obtained result before proceding.')
    logger.warning('Time elapsed to complete execution: '+str(end_test)+' seconds.')
    logger.warning('GOOD TO GO. Beware of ammount of API queries and cost depending on query type.')
    

except:
    logger.warning('ERROR: VERIFY INPUT FILE, DATA FORMAT, AND API KEY')

Distance Matrix Basic results for: 
ORIGEN_ID       450A
ORIG_LAT    -3.44508
ORIG_LONG   -79.9621
DEST_ID         452A
DEST_LAT    -3.47856
DEST_LONG   -80.2361
Name: 0, dtype: object
 are: 
{'destination_addresses': ['Tungurahua, Cantón Huaquillas, Ecuador'], 'origin_addresses': ['Cristóbal Colón 13-30, Santa Rosa, Ecuador'], 'rows': [{'elements': [{'distance': {'text': '47.0 km', 'value': 47004}, 'duration': {'text': '45 mins', 'value': 2712}, 'status': 'OK'}]}], 'status': 'OK'}
---
Distance Matrix Advance results for: 
ORIGEN_ID       450A
ORIG_LAT    -3.44508
ORIG_LONG   -79.9621
DEST_ID         452A
DEST_LAT    -3.47856
DEST_LONG   -80.2361
Name: 0, dtype: object
 are: 
{'destination_addresses': ['Tungurahua, Cantón Huaquillas, Ecuador'], 'origin_addresses': ['Cristóbal Colón 13-30, Santa Rosa, Ecuador'], 'rows': [{'elements': [{'distance': {'text': '47.0 km', 'value': 47004}, 'duration': {'text': '45 mins', 'value': 2712}, 'duration_in_traffic': {'text': '44 mins', 'value': 2650

Run the **Distance Matrix Basic** procedure for each haul in the list. Outputs are exported to a .xslx file containing original data and distance, time, and status. If you want to run the Distance Matrix Advance procedure, specify mode, departure time and traffic model in the query where comments specify.

In [6]:
index=Datos.index.tolist()
results=[] #Create an empty list to save results
start_time = time.time() #Start counting the time of the procedure
errors=0 #Variable to keep track of errors
for i in index: #We iterate over the index of the dataframe
    
    #Obtain all the information of the current register in dataframe
    current_register=Datos.loc[i].tolist() #Transform the current register into a list to handle easier.
    current_OrigID=current_register[0]     #Obtain the information of each column of the register.
    current_orig_lat=current_register[1]
    current_orig_long=current_register[2]
    current_DestID=current_register[3]
    current_dest_lat=current_register[4]
    current_dest_long=current_register[5]
    
    
    #Assign the current origin and destination points to tuples for Google Maps API input arguments
    orig=(current_orig_lat,current_orig_long)
    dest=(current_dest_lat,current_dest_long)
    
    #Run queries and update results
    try:
        current_result = gmaps.distance_matrix(orig, dest)# To run DM ADVANCE INCLUDE: , mode='driving', departure_time=now, traffic_model='best_guess') after dest.
        result_dict={'Distance_[meters]':current_result['rows'][0]['elements'][0]['distance']['value'],
                 'Duration_Time_[seconds]':current_result['rows'][0]['elements'][0]['duration']['value'],
                 #'Duration_In_Traffic_[seconds]':current_result['rows'][0]['elements'][0]['duration_in_traffic']['value'], #IF YOU RUN DM ADVANCE, INCLUDE THIS ROW BY DELETING THE INITIAL #
                 'Status':current_result['rows'][0]['elements'][0]['status']}
                 
    
        results.append(result_dict)
    
        results[i].update({'ORIGEN_ID': current_OrigID})
        results[i].update({'ORIG_LAT': current_orig_lat})
        results[i].update({'ORIG_LON': current_orig_long})
        results[i].update({'DEST_ID': current_DestID})
        results[i].update({'DEST_LAT': current_dest_lat})
        results[i].update({'DEST_LONG': current_dest_long})
        logger.warning('Distance for origin: '+str(current_OrigID)+' and dest:'+str(current_DestID)+' computed successfully')
    
    #If a query outputs an error, it is recorded in the corresponding register
    except:
        result_dict={'Distance_[meters]':'ERROR',
                 'Duration_Time_[seconds]':'ERROR',
                 #'Duration_In_Traffic_[seconds]':'ERROR',  #IF YOU RUN DM ADVANCE, INCLUDE THIS ROW BY DELEATING THE INITIAL #
                 'Status':'ERROR'}
        results.append(result_dict)
    
        results[i].update({'ORIGEN_ID': current_OrigID})
        results[i].update({'ORIG_LAT': current_orig_lat})
        results[i].update({'ORIG_LON': current_orig_long})
        results[i].update({'DEST_ID': current_DestID})
        results[i].update({'DEST_LAT': current_dest_lat})
        results[i].update({'DEST_LONG': current_dest_long})
        logger.warning('ERROR IN INPUT DATA, VERIFY RESULT')
        errors+=1

logger.warning('ALL HAULS MEASURED SUCCESSFULLY')
logger.warning('NUMBER OF ERRORS FOUND: '+str(errors)+'. BEWARE TO SOLVE THEM IN THE OUTPUT FILE!')
pd.DataFrame(results).to_excel(file_output, encoding='utf-8')
logger.warning('OUTPUT FILE GENERATED')
end_time=(time.time() - start_time)
logger.warning('TIME TO COMPUTE '+str(i+1)+' HAULS: '+str(end_time/60)+' MINUTES, OR '+str(end_time)+' SECONDS')
logger.warning('PROCESS FINISHED.')

Distance for origin: 450A and dest:452A computed successfully
Distance for origin: 804A and dest:805A computed successfully
Distance for origin: 807A and dest:841A computed successfully
Distance for origin: 543A and dest:111A computed successfully
Distance for origin: 165A and dest:111A computed successfully
Distance for origin: 804A and dest:805A computed successfully
Distance for origin: 365A and dest:734A computed successfully
Distance for origin: 334A and dest:322A computed successfully
Distance for origin: 806A and dest:841A computed successfully
Distance for origin: 550A and dest:552A computed successfully
Distance for origin: 328A and dest:354A computed successfully
Distance for origin: 348A and dest:322A computed successfully
Distance for origin: 845A and dest:781A computed successfully
Distance for origin: 346A and dest:328A computed successfully
Distance for origin: 148A and dest:257A computed successfully
Distance for origin: 354A and dest:346A computed successfully
Distance

## End of code.